# Movie Recommendation Systems Coursera 

In [1]:
import pandas as pd
from math import sqrt
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

In [2]:
path1  = 'https://github.com/krishnaik06/Recommendation_complete_tutorial/blob/master/KNN%20Movie%20Recommendation/movies.csv?raw=True'
path2 =  'https://github.com/krishnaik06/Recommendation_complete_tutorial/blob/master/KNN%20Movie%20Recommendation/ratings.csv?raw=True'

In [3]:

movies_df = pd.read_csv(path1)
rating_df=pd.read_csv(path2)    #,usecols=['userId', 'movieId', 'rating'],
 #   dtype={'userId': 'int32', 'movieId': 'int32', 'rating': 'float32'})

In [4]:
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
rating_df.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [6]:
#movies_df = pd.read_csv(path1)
#movies_df.head()
#ratings_df = pd.read_csv(path2)
#ratings_df.head()

def get_year(string):         # function to extract date from movie titles
    return( string[-5:-1])
def del_year(string):
    return(string[:-7])
movies_df['year']= movies_df['title'].apply(get_year)
movies_df['title']= movies_df['title'].apply(del_year)
movies_df['genres'] = movies_df.genres.str.split('|')
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


In [7]:
movies_with_genres_df = movies_df.copy()
for index, row in movies_df.iterrows():
    for genre in row['genres']:
        movies_with_genres_df.at[index,genre] =1
        
        
movies_with_genres_df.fillna(0,inplace =True)
movies_with_genres_df.head()


,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,"[Comedy, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II,[Comedy],1995,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
rating_df.drop('timestamp',axis =1,inplace =True)
rating_df.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [9]:
userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

,rating,title
0,5.0,"Breakfast Club, The"
1,3.5,Toy Story
2,2.0,Jumanji
3,5.0,Pulp Fiction
4,4.5,Akira


In [10]:
inputid = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]
inputmovies = pd.merge(inputMovies,inputid)
inputmovies = inputmovies.drop('genres', 1).drop('year', 1)
inputmovies


,rating,title,movieId
0,5.0,"Breakfast Club, The",1968
1,3.5,Toy Story,1
2,2.0,Jumanji,2
3,5.0,Pulp Fiction,296
4,4.5,Akira,1274


In [12]:
userMovies = movies_with_genres_df[movies_with_genres_df['movieId'].isin(inputmovies['movieId'].tolist())]
userMovies

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
257,296,Pulp Fiction,"[Comedy, Crime, Drama, Thriller]",1994,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
973,1274,Akira,"[Action, Adventure, Animation, Sci-Fi]",1988,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1445,1968,"Breakfast Club, The","[Comedy, Drama]",1985,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
userMovies = userMovies.reset_index(drop=True)
#Dropping unnecessary issues due to save memory and to avoid issues
userGenreTable = userMovies.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
userGenreTable

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
inputmovies['rating']

0    5.0
1    3.5
2    2.0
3    5.0
4    4.5
Name: rating, dtype: float64

In [15]:
userGenreTable.transpose()

,0,1,2,3,4
Adventure,1.0,1.0,0.0,1.0,0.0
Animation,1.0,0.0,0.0,1.0,0.0
Children,1.0,1.0,0.0,0.0,0.0
Comedy,1.0,0.0,1.0,0.0,1.0
Fantasy,1.0,1.0,0.0,0.0,0.0
Romance,0.0,0.0,0.0,0.0,0.0
Drama,0.0,0.0,1.0,0.0,1.0
Action,0.0,0.0,0.0,1.0,0.0
Crime,0.0,0.0,1.0,0.0,0.0
Thriller,0.0,0.0,1.0,0.0,0.0


In [16]:
userProfile = userGenreTable.transpose().dot(inputmovies['rating'])
#The user profile
userProfile

Adventure             13.5
Animation             10.0
Children               8.5
Comedy                11.5
Fantasy                8.5
Romance                0.0
Drama                  6.5
Action                 5.0
Crime                  2.0
Thriller               2.0
Horror                 0.0
Mystery                0.0
Sci-Fi                 5.0
War                    0.0
Musical                0.0
Documentary            0.0
IMAX                   0.0
Western                0.0
Film-Noir              0.0
(no genres listed)     0.0
dtype: float64

In [17]:
genreTable = movies_with_genres_df.set_index(movies_with_genres_df['movieId'])
#And drop the unnecessary information
genreTable = genreTable.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
genreTable.head()

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
movieId,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
#(genreTable*userProfile).sum(axis=1)
userProfile.sum()

72.5

In [22]:
recommendationTable_df = ((genreTable*userProfile).sum(axis=1))/(userProfile.sum())
recommendationTable_df.head()

movieId
1    0.717241
2    0.420690
3    0.158621
4    0.248276
5    0.158621
dtype: float64

In [23]:
recommendationTable_df = recommendationTable_df.sort_values(ascending=False)
#Just a peek at the values
recommendationTable_df.head()

movieId
134853    0.806897
130520    0.786207
32031     0.786207
26340     0.786207
51939     0.786207
dtype: float64

In [73]:
movies_df.loc[movies_df['movieId'].isin(recommendationTable_df.head(20).keys())]

,movieId,title,genres,year
478,546,Super Mario Bros.,"[Action, Adventure, Children, Comedy, Fantasy,...",1993
559,673,Space Jam,"[Adventure, Animation, Children, Comedy, Fanta...",1996
2250,2987,Who Framed Roger Rabbit?,"[Adventure, Animation, Children, Comedy, Crime...",1988
2355,3114,Toy Story 2,"[Adventure, Animation, Children, Comedy, Fantasy]",1999
3568,4886,"Monsters, Inc.","[Adventure, Animation, Children, Comedy, Fantasy]",2001
5490,26340,"Twelve Tasks of Asterix, The (Les douze travau...","[Action, Adventure, Animation, Children, Comed...",1976
5819,32031,Robots,"[Adventure, Animation, Children, Comedy, Fanta...",2005
6047,40339,Chicken Little,"[Action, Adventure, Animation, Children, Comed...",2005
6260,47124,"Ant Bully, The","[Adventure, Animation, Children, Comedy, Fanta...",2006
6448,51939,TMNT (Teenage Mutant Ninja Turtles),"[Action, Adventure, Animation, Children, Comed...",2007


In [5]:
df = pd.merge( rating_df,movies_df, on = 'movieId')
df.head()

,userId,movieId,rating,title
0,1,1,4.0,Toy Story (1995)
1,5,1,4.0,Toy Story (1995)
2,7,1,4.5,Toy Story (1995)
3,15,1,2.5,Toy Story (1995)
4,17,1,4.5,Toy Story (1995)


In [6]:
total_rating = df.groupby('title')['rating'].count()
total_ratings = pd.DataFrame(total_rating)
total_ratings.rename(columns ={'rating':'total_rating'},inplace =True)
total_ratings.head()

,total_rating
title,
'71 (2014),1
'Hellboy': The Seeds of Creation (2004),1
'Round Midnight (1986),2
'Salem's Lot (2004),1
'Til There Was You (1997),2


In [7]:
df_final = pd.merge(df,total_ratings, on = 'title')
df_final.head()

,userId,movieId,rating,title,total_rating
0,1,1,4.0,Toy Story (1995),215
1,5,1,4.0,Toy Story (1995),215
2,7,1,4.5,Toy Story (1995),215
3,15,1,2.5,Toy Story (1995),215
4,17,1,4.5,Toy Story (1995),215


In [8]:
minimum_votes = 50 #randomly decided
df_combined = df_final[df_final['total_rating']>50]
df_combined.head()

,userId,movieId,rating,title,total_rating
0,1,1,4.0,Toy Story (1995),215
1,5,1,4.0,Toy Story (1995),215
2,7,1,4.5,Toy Story (1995),215
3,15,1,2.5,Toy Story (1995),215
4,17,1,4.5,Toy Story (1995),215


In [9]:
movie_mat = df_combined.pivot_table(index = 'title', columns = 'userId',values = 'rating').fillna(0)
movie_mat.head()

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
10 Things I Hate About You (1999),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
12 Angry Men (1957),0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2001: A Space Odyssey (1968),0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,5.0,0.0,3.0,0.0,4.5
28 Days Later (2002),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,5.0
300 (2007),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,3.0,0.0,0.0,5.0,0.0,4.0


In [17]:
from scipy.sparse import csr_matrix

movie_mat_matrix = csr_matrix(movie_mat.values)
list_metrics = ['cityblock', 'cosine', 'euclidean', 'l1', 'l2','manhattan','braycurtis', 'canberra', 'chebyshev',
      'correlation', 'dice', 'hamming', 'jaccard', 'kulsinski',
      'mahalanobis', 'minkowski', 'rogerstanimoto', 'russellrao',
      'seuclidean', 'sokalmichener', 'sokalsneath', 'sqeuclidean',
      'yule']
from sklearn.neighbors import NearestNeighbors

for i in list_metrics:
    model_knn = NearestNeighbors(metric = i)         #metric was 'cosine' erlier
    model_knn.fit(movie_mat_matrix)
    query_index =416 #np.random.choice(movie_mat.shape[0])
    
    distances, indices = model_knn.kneighbors(movie_mat.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)
    for i in range(0, len(distances.flatten())):#Evaluation usin cosine distance
        if i == 0:
            print('Recommendations for {0}:\n'.format(movie_mat.index[query_index]))
        else:
            print('{0}: {1}, with distance of {2}:'.format(i, movie_mat.index[indices.flatten()[i]], distances.flatten()[i]))
model_knn = NearestNeighbors(metric = 'euclidean')         #metric was 'cosine' erlier
model_knn.fit(movie_mat_matrix)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='euclidean',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [24]:
query_index =416 #np.random.choice(movie_mat.shape[0])
print(query_index)
distances, indices = model_knn.kneighbors(movie_mat.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)

416


In [12]:
for i in range(0, len(distances.flatten())):#Evaluation usin cosine distance
    if i == 0:
        print('Recommendations for {0}:\n'.format(movie_mat.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, movie_mat.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for Wallace & Gromit: The Wrong Trousers (1993):

1: Chicken Run (2000), with distance of 0.52260422706604:
2: Monty Python and the Holy Grail (1975), with distance of 0.5330187082290649:
3: Monty Python's Life of Brian (1979), with distance of 0.5862387418746948:
4: Toy Story 2 (1999), with distance of 0.5907834768295288:
5: Indiana Jones and the Temple of Doom (1984), with distance of 0.598820149898529:


In [19]:
for i in range(0, len(distances.flatten())): #Evaluation usin euclidean distance
    if i == 0:
        print('Recommendations for {0}:\n'.format(movie_mat.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, movie_mat.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for Wallace & Gromit: The Wrong Trousers (1993):

1: Crocodile Dundee (1986), with distance of 31.416555404663086:
2: Splash (1984), with distance of 31.42451286315918:
3: Wild Wild West (1999), with distance of 31.432466506958008:
4: Honey, I Shrunk the Kids (1989), with distance of 31.674121856689453:
5: Lost World: Jurassic Park, The (1997), with distance of 32.345787048339844:


## As we can see that using a different metric can significantly change the recommended movies therefore emphasis should be made on which metric to use.

In [26]:
type(distances.flatten())

numpy.ndarray

In [16]:
type(query_index)

int

In [32]:
list_metrics_sparse = ['cityblock', 'cosine', 'euclidean', 'l1', 'l2','manhattan']
list_metrics=['braycurtis', 'canberra', 'chebyshev',
      'correlation', 'dice', 'hamming', 'jaccard', 'kulsinski',
       'minkowski', 'rogerstanimoto', 'russellrao']
from sklearn.neighbors import NearestNeighbors

for i in list_metrics_sparse:
    model_knn = NearestNeighbors(metric = i)         #metric was 'cosine' erlier
    model_knn.fit(movie_mat_matrix)
    query_index =416 #np.random.choice(movie_mat.shape[0])
    
    distances, indices = model_knn.kneighbors(movie_mat.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)
    print('****** {} *******'.format(i))
    for i in range(0, len(distances.flatten())):#Evaluation usin cosine distance
        if i == 0:
            print('Recommendations for {0}:\n'.format(movie_mat.index[query_index]))
            
        else:
            print('{0}: {1}, with distance of {2}:'.format(i, movie_mat.index[indices.flatten()[i]], distances.flatten()[i]))

print("**************************************************************************************************")
print("***********************************for non sparsed data********************************************")
print("**************************************************************************************************")
            
for i in list_metrics:
    model_knn = NearestNeighbors(metric = i)         #metric was 'cosine' erlier
    model_knn.fit(movie_mat.values)
    query_index =416 #np.random.choice(movie_mat.shape[0])
    
    distances, indices = model_knn.kneighbors(movie_mat.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)
    print('****** {} *******'.format(i))
    for i in range(0, len(distances.flatten())):#Evaluation usin cosine distance
        if i == 0:
            print('Recommendations for {0}:\n'.format(movie_mat.index[query_index]))
            
        else:
            print('{0}: {1}, with distance of {2}:'.format(i, movie_mat.index[indices.flatten()[i]], distances.flatten()[i]))            

****** cityblock *******
Recommendations for Wallace & Gromit: The Wrong Trousers (1993):

1: Splash (1984), with distance of 265.0:
2: Crocodile Dundee (1986), with distance of 267.0:
3: Lady and the Tramp (1955), with distance of 274.0:
4: Wild Wild West (1999), with distance of 275.0:
5: Rocky Horror Picture Show, The (1975), with distance of 280.5:
****** cosine *******
Recommendations for Wallace & Gromit: The Wrong Trousers (1993):

1: Chicken Run (2000), with distance of 0.5226041078567505:
2: Monty Python and the Holy Grail (1975), with distance of 0.5330187082290649:
3: Monty Python's Life of Brian (1979), with distance of 0.5862388014793396:
4: Toy Story 2 (1999), with distance of 0.5907834768295288:
5: Indiana Jones and the Temple of Doom (1984), with distance of 0.598820149898529:
****** euclidean *******
Recommendations for Wallace & Gromit: The Wrong Trousers (1993):

1: Crocodile Dundee (1986), with distance of 31.416555404663086:
2: Splash (1984), with distance of 31.42